## Calculate the equatorial coordinates (right ascension and declination) of the events + make a skymap of the reconstructed directions of the events in these coordinates.

In [3]:
def utc(df):   #function that calculates the utc time of the events  
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)
        
    return utc_time

In [4]:
def equatorial_coords(df):   #function that calculates the equatorial coordinates (ra and dec) of the events from the horizontal coordinates 
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)

    df = df.assign(utc_time = utc_time)   #add new column to the data frame with the UTC observation times of the events

    loc = EarthLocation(lat = 28.76152611*u.deg, lon = -17.89149701*u.deg, height = 2184*u.m)   #location of the telescope (CTA LST1: Roque de los Muchachos)

    hor_coords = SkyCoord(alt = df['reco_alt'], az = df['reco_az'], frame = 'altaz', unit = 'rad', 
                          obstime = df['utc_time'], location = loc)   #horizontal coordinates of the events
    eq_coords = hor_coords.icrs   #equatorial coordinates of the events
    
    return eq_coords

In [ ]:
data = tot_data[(tot_data['gammaness']>0.8)]

eq_coords = equatorial_coords(data)
data = data.assign(utc_time = utc(data))   #add new column to the data frame with the UTC observation times of the events
data = data.assign(RA = eq_coords.ra)   #add columns with right ascension and declination (equatorial coordinates) of the events
data = data.assign(DEC = eq_coords.dec)

# skymap (RA, dec) de las posiciones reconstruidas de los sucesos (ya se ha aplicado el corte de gammaness > 0.8):
counts, bins_x, bins_y = np.histogram2d(data['RA'], data['DEC'], bins=250)
fig = figure(x_range=(min(bins_x), max(bins_x)), y_range=(min(bins_y), max(bins_y)), plot_width=500, plot_height=300)
fig.image(image=[counts], x=bins_x[0], y=bins_y[0], dw=bins_x[-1] - bins_x[0], dh=bins_y[-1] - bins_y[0], palette=Inferno256) 
fig.xaxis.axis_label = 'right ascension (deg)'
fig.yaxis.axis_label = 'declination (deg)'
color_mapper = LinearColorMapper(palette=Inferno256, low=counts.min(), high=counts.max())
color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(), location=(0,0))
fig.add_layout(color_bar, 'right')
show(fig)